In [2]:
from __future__ import division
from gurobipy import *
import numpy as np
import os
import csv
import seaborn as sns
import scipy as sp
import operator
import psycopg2
import folium as flm
import string
import mysql.connector as mdb
import pandas as pd
import json
import datetime
import folium
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.rc('text', usetex=False)
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams.update({'legend.fontsize': 16})



from IPython.display import clear_output
from IPython.display import HTML
from matplotlib.ticker import MultipleLocator
from heapq import merge


def inline_map(map1):
    map1._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 600px; border: none"></iframe>'.format(srcdoc=map1.HTML.replace('"', '&quot;')))

def embed_map(map1, path="map.html"):
    map1.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 600px; border: none"></iframe>'.format(path=path))




In [3]:
import socket
socket_name=socket.gethostname()
if socket_name=='emres-MBP':
    os.system("ssh -p 5022 -f -N -L localhost:5433:localhost:5432 emre@bgrid.lbl.gov")
    os.system("ssh -f -N -L localhost:8800:localhost:3306 emre@bt-eplus.dhcp.lbl.gov")
else: 
    os.system("ssh -p 5022 -f -N -L localhost:5433:localhost:5432 emre@bgrid.lbl.gov")

In [3]:
google_ev=pd.read_pickle('google_topflow_dweight.pkl')
google_cdr=pd.read_pickle('google_topflow.pkl')

In [9]:
list(google_ev['origin_zip'])+[94043]

[95051, 94085, 95035, 95129, 94539, 94043]

In [5]:
google_cdr

,origin_zip,flow,distance,route_distance,time,traffic_time,weighted_dist,p(distance),flow_dweight
49,94085,4.512549,5.72099,8.1594,472,525,1.012977,0.011601,0.052352
47,94087,1.951191,7.07228,9.5627,556,581,0.513334,0.011601,0.022637
6,94040,1.669936,3.22095,3.8608,363,411,0.177377,0.011601,0.019374
169,94539,1.447528,17.97180,27.5423,1297,1300,1.096848,0.023203,0.033587
26,95051,1.359786,10.39980,14.9363,884,954,0.558770,0.041434,0.056341


In [10]:
google_ev_zips=list(google_ev['origin_zip'])+[94043]
google_cdr_zips=list(google_cdr['origin_zip'])+[94043]

In [10]:
import psycopg2
# connect over this tunnel 
conn = psycopg2.connect(database="bgrid", user="emre", password='Kara1234', host="localhost", port="5433")


# Open a cursor to perform database operations
cur = conn.cursor()

zip_codes_visited=np.array(google_ev_zips)
visited_zips=str(np.array_str(zip_codes_visited)).replace('[','').replace(']','').replace(' ',',')

# GETTING POLYGON DATA FROM SWITCH-DB2 and WRITE IT TO A LOCAL JSON FILE FOR FOLIUM

getJSON='''SELECT row_to_json(fc)
FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type
   , ST_AsGeoJSON(lg.geom)::json As geometry
    , row_to_json(lp) As properties
  FROM gis_data.census_zcta As lg
       INNER JOIN (SELECT aland10, geoid10 FROM gis_data.census_zcta where CAST (geoid10 AS INTEGER) IN ({})) As lp
      ON lg.geoid10 = lp.geoid10  ) As f )  As fc;'''.format(visited_zips)
cur.execute(getJSON)

vals=cur.fetchall()
json_to_write=json.dumps(vals[0][0])
f = open('./google_ev_zips.json','w')
f.write(json_to_write) 
f.close()
conn.close()

OperationalError: FATAL:  password authentication failed for user "emre"
FATAL:  password authentication failed for user "emre"


In [22]:
# SUPLAP COLOR CODED PLOT
map = folium.Map(location=[37.535238, -122.201525], zoom_start=10,tiles='Stamen Terrain')
from itertools import cycle
col_gen =['#7CB9E8','#FF7E00','#8DB600','#6E7F80','#FFE135','magenta','#553592','#0047AB','#black','#C53151','#7C0A02','cyan']
ct=0
map.geo_json(geo_path='./google_ev_zips.json',
             fill_color=col_gen[ct], fill_opacity=0.85, line_opacity=0.7)
map.create_map(path='mtn_ev_map.html')
embed_map(map,path='mtn_ev_map.html')

In [1]:
range(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]